![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/arc/images/microsoft-small-logo.png)
 
## Create a PostgreSQL server - Azure Arc on an existing Azure Arc Data Controller
 
This notebook walks through the process of creating a PostgreSQL server - Azure Arc on an existing Azure Arc Data Controller.
 
* Follow the instructions in the **Prerequisites** cell to install the tools if not already installed.
* Make sure you have the target Azure Arc Data Controller already created.

<span style="color:red"><font size="3">Please press the "Run All" button to run the notebook</font></span>

### **Prerequisites** 
Ensure the following tools are installed and added to PATH before proceeding.
 
|Tools|Description|Installation|
|---|---|---|
|Azure CLI (az) | Command-line tool for installing and managing resources in an Azure Arc cluster |[Installation](https://docs.microsoft.com/cli/azure/install-azure-cli-windows?tabs=azure-cli) |
|Azure CLI arcdata extension | Commands for using Azure Arc for Azure data services. | [Installation](https://docs.microsoft.com/azure/azure-arc/data/install-arcdata-extension)

### **Setup and Check Prerequisites**

In [ ]:
import sys,os,json,subprocess
def run_command():
    print("Executing: " + cmd)
    output = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True )
    if output.returncode != 0:
        print(f'Command: {cmd} failed \n')
        print(f'\t>>>Error output: {output.stderr.decode("utf-8")}\n')
        sys.exit(f'exit code: {output.returncode}\n')
    print(f'Successfully executed: {cmd}')
    print(f'\t>>>Output: {output.stdout.decode("utf-8")}\n')
    return output.stdout.decode("utf-8")
cmd = 'az --version'
out = run_command()


### **Set variables**

#### 

Generated by Azure Data Studio using the values collected in the 'Deploy a PostgreSQL server - Azure Arc instance' wizard

### **Creating the PostgreSQL server - Azure Arc instance**

In [ ]:
print (f'Creating the PostgreSQL server - Azure Arc instance')

port = f' --port "{postgres_server_port}"' if postgres_server_port else ""
service_type = f' --service-type "{postgres_server_service_type}"' if postgres_server_service_type else ""
volume_size_data = f' --volume-size-data {postgres_server_volume_size_data}Gi' if postgres_server_volume_size_data else ""
volume_size_logs = f' --volume-size-logs {postgres_server_volume_size_logs}Gi' if postgres_server_volume_size_logs else ""

storage_class_data = f' --storage-class-data {postgres_storage_class_data}' if postgres_storage_class_data else ""
storage_class_logs = f' --storage-class-logs {postgres_storage_class_logs}' if postgres_storage_class_logs else ""

cores_request = f' --cores-request {postgres_server_cores_request}' if postgres_server_cores_request else ""
cores_limit = f' --cores-limit {postgres_server_cores_limit}' if postgres_server_cores_limit else ""
memory_request = f' --memory-request {postgres_server_memory_request}Gi' if postgres_server_memory_request else ""
memory_limit = f' --memory-limit {postgres_server_memory_limit}Gi' if postgres_server_memory_limit else ""
dev_use = ' --dev' if postgres_server_dev_use == 'true' else ''

os.environ["AZDATA_USERNAME"] = postgres_server_username
os.environ["AZDATA_PASSWORD"] = os.environ["AZDATA_NB_VAR_POSTGRES_SERVER_PASSWORD"]
cmd = f'az postgres server-arc create --name {postgres_server_name} --k8s-namespace {arc_data_controller_namespace} --use-k8s {storage_class_data}{storage_class_logs}{port}{service_type}{volume_size_data}{volume_size_logs}{cores_request}{cores_limit}{memory_request}{memory_limit}{dev_use}'
out=run_command()